In [138]:
import urllib
import json
import pandas as pd

In [140]:
step4_df = pd.read_csv('step4_df.csv')
step4_df.head()

,corp_name,corp_code,bsns_year,reprt_code,data
0,LG파워콤,318617,2020,11011,"{""status"":""013"",""message"":""조회된 데이타가 없습니다.""}"
1,LG파워콤,318617,2020,11012,"{""status"":""013"",""message"":""조회된 데이타가 없습니다.""}"
2,LG파워콤,318617,2019,11011,"{""status"":""013"",""message"":""조회된 데이타가 없습니다.""}"
3,LG파워콤,318617,2019,11012,"{""status"":""013"",""message"":""조회된 데이타가 없습니다.""}"
4,LG파워콤,318617,2018,11011,"{""status"":""013"",""message"":""조회된 데이타가 없습니다.""}"


In [161]:
step5_df = pd.DataFrame(columns=["corp_name","bsns_year","reprt_code", "sexdstn","fo_bbm","rgllbr_co", "avrg_cnwk_sdytrn", "jan_salary_am"])

print("calculating {} items...".format(len(step4_df)))
for index, row in step4_df.iterrows():
    data = json.loads(row["data"])
    if data['status'] != '000':
        continue
    
    new_row = {
        "corp_name": row["corp_name"],
        "bsns_year": row["bsns_year"],
        "reprt_code": row["reprt_code"],
    }
    
    for item in data['list']:
        try:
            new_row["sexdstn"] = item["sexdstn"]
            new_row["fo_bbm"] = item["fo_bbm"]
            
            rgllbr_co = item["rgllbr_co"].replace(",", "").strip()
            new_row["rgllbr_co"] = int(rgllbr_co) if rgllbr_co.isdigit() else 0

            avrg_cnwk_sdytrn = item["avrg_cnwk_sdytrn"].strip().replace('.', '', 1)
            new_row["avrg_cnwk_sdytrn"] = float(item["avrg_cnwk_sdytrn"].strip()) if avrg_cnwk_sdytrn.isdigit() else 0
            
            jan_salary_am = item["jan_salary_am"].replace(",", "").strip()
            new_row["jan_salary_am"] = int(jan_salary_am) if jan_salary_am.isdigit() else 0
            
            step5_df = step5_df.append(new_row, ignore_index=True)
        except:
            print(json.dumps(item, ensure_ascii=False, indent=2))

step5_df.to_csv('step5_df.csv', index=False)
print("done")

calculating 168 items...
done


In [162]:
step5_df[step5_df['corp_name'] == "LG화학"].head()

,corp_name,bsns_year,reprt_code,sexdstn,fo_bbm,rgllbr_co,avrg_cnwk_sdytrn,jan_salary_am
426,LG화학,2020,11012,남,석유화학,5238,14.1,51000000
427,LG화학,2020,11012,여,공통 및 기타부문,623,7.12,34000000
428,LG화학,2020,11012,여,석유화학,420,8.11,31000000
429,LG화학,2020,11012,남,전지,5573,8.08,41000000
430,LG화학,2020,11012,여,전지,920,6.03,32000000


In [163]:
import numpy as np

def weight_sum(s, df, weight_column, round_digit=0):
    try:
        value = round(np.average(s, weights=df.loc[s.index, weight_column]), round_digit)
        return int(value) if round_digit == 0 else value
    except:
        return 0

#step5_df = step5_df[(step5_df['corp_name'] == "LG화학") & (step5_df['bsns_year'] == 2016)]

step5_total = step5_df.groupby(['corp_name', 'bsns_year', 'reprt_code', 'sexdstn']).agg({
    'rgllbr_co': 'sum',
    'avrg_cnwk_sdytrn': lambda s: weight_sum(s, step5_df, 'rgllbr_co', 2),
    'jan_salary_am': lambda s: weight_sum(s, step5_df, 'rgllbr_co'),

})

step5_total = step5_total.reset_index()
step5_total.to_csv('step5_total.csv', index=False)

step5_total.head()

,corp_name,bsns_year,reprt_code,sexdstn,rgllbr_co,avrg_cnwk_sdytrn,jan_salary_am
0,LG,2015,11011,남,84,3.7,100709000
1,LG,2015,11011,여,25,5.1,53253000
2,LG,2015,11012,남,0,0.0,0
3,LG,2015,11012,여,0,0.0,0
4,LG,2016,11011,남,95,5.0,106403000
